# Isotonic Convex Quantile Regression (ICQR)

- Author: Sheng Dai (sheng.dai@aalto.fi)
- Date  : July 30, 2020

Reference:

[1] Kuosmanen T., Dai S., Zhou X. Partial frontiers are not quantiles. Unpublished working paper (available from the authors by request).

In [1]:
import pandas as pd
import numpy as np

In [2]:
# import the package pystoned
from pystoned import ICQER

In [3]:
# import Finnish electricity distribution firms data
url = 'https://raw.githubusercontent.com/ds2010/pyStoNED-Tutorials/master/Data/firms.csv'
df = pd.read_csv(url, error_bad_lines=False)

In [4]:
# output
y = df['Energy']

# inputs
x1 = df['OPEX']
x1 = np.asmatrix(x1).T
x2 = df['CAPEX']
x2 = np.asmatrix(x2).T
x = np.concatenate((x1, x2), axis=1)

In [5]:
# define and solve the ICQR model

tau = 0.5
cet = "addi"
fun = "prod"
rts = "vrs"

model = ICQER.icqr(y, x, tau, cet, fun, rts)

# using local solver (MOSEK API)
from pyomo.opt import SolverFactory
opt = SolverFactory("mosek")
results = opt.solve(model, tee=True)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization problem)
  Constraints            : 3714            
  Cones                  : 0               
  Scalar variables       : 534             
  Matrix variables       : 0               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.02    
Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : LO (linear optimization 

In [6]:
# retrive the residuals (eplus)
val = list(model.ep[:].value)
eplus = np.asarray(val)
eplus

array([  0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,  84.8856305 ,   0.        ,   0.        ,
         0.        ,  32.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,  93.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,  29.        ,   0.        ,   7.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
        70.        ,   0.        ,   0.        , 224.        ,
        34.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,  11.89142857,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         2.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,  10.89142857,   0.        ,
         0.        ,   0.        ,   0.        ,  77.6808952 ,
        86.        ,  32.        ,   0.        ,   0.        ,
         5.        ,   0.        , 256.8239392 ,   0.  

In [7]:
# retrive the residuals (eminus)
val = list(model.em[:].value)
eminus = np.asarray(val)
eminus

array([  0.        ,   0.        ,  20.        , 495.81692573,
         1.10857143,   0.        ,   9.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   2.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,  10.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   3.        ,  23.        , 197.4119171 ,
         0.        ,   6.        ,   4.        ,   0.        ,
         0.        ,   0.        ,   0.        , 152.        ,
        39.1143695 ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   3.10857143,   0.        ,   0.        ,
         1.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,  54.        ,
         0.        , 245.        ,   0.        ,   0.  

In [8]:
# retrive the beta
ind = list(model.b)
val = list(model.b[:,:].value)
beta= np.asarray([ i + tuple([j]) for i, j in zip(ind, val)]) 

beta = pd.DataFrame(beta, columns = ['Name', 'Key', 'Value'])
beta = beta.pivot(index='Name', columns='Key', values='Value')
beta.columns = ['b1', 'b2']

beta

,b1,b2
Name,,
0.0,2.597403e-02,0.000000e+00
1.0,5.604070e-02,3.212089e-02
2.0,0.000000e+00,0.000000e+00
3.0,0.000000e+00,0.000000e+00
4.0,0.000000e+00,0.000000e+00
...,...,...
84.0,0.000000e+00,0.000000e+00
85.0,0.000000e+00,-2.225388e-17
86.0,2.500000e-02,0.000000e+00
